In [ ]:
# === 安裝必要套件（Colab 建議保留）===
!pip install -q tensorflow tensorflow_examples matplotlib pillow

# === 匯入套件 ===
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow_examples.models.pix2pix import pix2pix

# Colab 介面
from google.colab import files, drive

# === 參數設定（可自行調整）===
DRIVE_IMAGE_PATH = "/content/drive/MyDrive/A32C58165B8BC0CDC12E54B723ED8A2C4569913E.jpg"  # 預設圖片路徑
DRIVE_MODEL_PATH = "/content/drive/MyDrive/my_pix2pix_model.h5"  # 預設模型權重（可留不存在）
LAMBDA_CYCLE = 0.8  # 可改 5.0 或 0.1

# === 輔助：掛載 Drive（必要時才掛）===
def ensure_drive_mounted():
    if not os.path.exists("/content/drive") or not os.path.ismount("/content/drive"):
        drive.mount('/content/drive', force_remount=True)

# === 1) 上傳檔案（可一次上傳：圖片 + 可選 .h5 權重）===
print("🔼 請上傳圖片（.jpg/.png/...）與可選的模型權重（.h5）。若不想上傳，請直接關閉/取消視窗。")
uploaded = files.upload()  # 若使用者取消，會回傳空 dict

image_path = None
model_path = None

if uploaded:
    # 依副檔名辨識
    img_exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
    h5_exts = {".h5"}

    uploaded_files = list(uploaded.keys())

    # 找圖片
    for fname in uploaded_files:
        ext = os.path.splitext(fname)[1].lower()
        if ext in img_exts:
            image_path = fname
            break

    # 找模型
    for fname in uploaded_files:
        ext = os.path.splitext(fname)[1].lower()
        if ext in h5_exts:
            model_path = fname
            break

    if image_path:
        print(f"✅ 使用上傳圖片：{image_path}")
    else:
        print("ℹ️ 未偵測到上傳圖片，改用 Google Drive 預設路徑。")

    if model_path:
        print(f"✅ 使用上傳模型權重：{model_path}")

# === 2) 若沒有上傳圖片則改用 Drive 圖片 ===
if image_path is None:
    ensure_drive_mounted()
    if not os.path.exists(DRIVE_IMAGE_PATH):
        raise FileNotFoundError(
            f"找不到預設圖片：{DRIVE_IMAGE_PATH}\n"
            "請上傳圖片，或把 DRIVE_IMAGE_PATH 改成正確路徑。"
        )
    image_path = DRIVE_IMAGE_PATH
    print(f"✅ 使用雲端圖片：{image_path}")

# === 3) 決定模型權重來源：上傳優先，其次 Drive，最後使用未訓練權重 ===
if model_path is None:
    # 嘗試用 Drive 權重
    ensure_drive_mounted()
    if os.path.exists(DRIVE_MODEL_PATH):
        model_path = DRIVE_MODEL_PATH
        print(f"✅ 使用雲端模型權重：{model_path}")
    else:
        print("ℹ️ 找不到模型權重或未設定 MODEL_PATH，將使用未訓練權重。")

# === 4) 讀取與前處理圖片 ===
img = Image.open(image_path).convert("RGB").resize((256, 256))
img = np.array(img, dtype=np.float32) / 255.0
input_image = tf.expand_dims(img, axis=0)  # [1, 256, 256, 3]

# === 5) 建立 pix2pix 生成器 → 視需要載入權重 ===
generator = pix2pix.unet_generator(output_channels=3)

if model_path is not None and os.path.exists(model_path):
    try:
        generator.load_weights(model_path)
        print(f"✅ 已載入模型權重：{model_path}")
    except Exception as e:
        print(f"⚠️ 載入權重失敗（改用未訓練權重繼續）：{e}")
else:
    # 明確提示（這不是錯誤）
    print("ℹ️ 未載入任何權重：使用未訓練權重進行示範。")

# === 6) 前向推論（未訓練模型僅示範流程）===
pix2pix_output = generator(input_image, training=False)

# === 7) 簡化版 cycle-consistency loss 與噪音模擬 ===
def cycle_loss(real_image, cycled_image, lambda_cycle: float):
    loss = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return lambda_cycle * loss

noise = tf.random.normal(
    shape=tf.shape(input_image), mean=0.0,
    stddev=1.0 / float(LAMBDA_CYCLE), dtype=input_image.dtype
)
generated_image = tf.clip_by_value(input_image + noise, 0.0, 1.0)

# === 8) 視覺化 ===
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1); plt.title("Original"); plt.imshow(tf.squeeze(input_image)); plt.axis('off')
plt.subplot(1, 3, 2); plt.title("pix2pix Output"); plt.imshow(tf.squeeze(tf.clip_by_value(pix2pix_output, 0.0, 1.0))); plt.axis('off')
plt.subplot(1, 3, 3); plt.title("Noise Added"); plt.imshow(tf.squeeze(generated_image)); plt.axis('off')
plt.show()

difference_map = tf.abs(tf.cast(pix2pix_output, generated_image.dtype) - generated_image)
plt.figure(figsize=(6, 6))
plt.title("pix2pix vs Noise Difference")
plt.imshow(tf.squeeze(tf.clip_by_value(difference_map, 0.0, 1.0)))
plt.axis('off')
plt.show()